In [ ]:
import sys
relative_root = "../.."
sys.path.append(relative_root)

In [ ]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
from examples.training import house_prices_config
from scipy.stats import skew
from pathlib import Path

In [ ]:
train = pd.read_csv(Path(relative_root) / house_prices_config.LOCAL_RAW_TRAIN_FILENAME)

In [ ]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])

#log transform skewed numeric features:
numeric_feats = train.dtypes[train.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

train[skewed_feats] = np.log1p(train[skewed_feats])

In [ ]:
train = pd.get_dummies(train)

In [ ]:
#filling NA's with the mean of the column:
train = train.fillna(train.mean())

In [ ]:
#creating matrices for sklearn:
features = [col for col in train.columns if col != "SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(
    train[features],
    train["SalePrice"],
    test_size=0.33,
    random_state=88,
)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
model_ridge = Ridge(alpha = 10, random_state=88).fit(X_train, y_train)

In [ ]:
predictions = model_ridge.predict(X_test)

In [ ]:
mean_squared_error(predictions, y_test, squared=False)

## Init trubrics context

In [ ]:
from trubrics.context import DataContext, ModelContext

In [ ]:
data_context = DataContext(
    name="house_prices_dataset",
    version=0.1,
    training_data=X_train.assign(SalePrice=y_train),
    testing_data=X_test.assign(SalePrice=y_test),
    target_column="SalePrice"
)
model_context = ModelContext(
    name="house_prices_model",
    version=0.1,
    estimator=model_ridge,
    evaluation_function=mean_squared_error,
    evaluation_kwargs={"squared": False}
)

## Init trubrics modeller

In [ ]:
from trubrics.modellers.regressor import Regressor

In [ ]:
trubrics_model = Regressor(data=data_context, model=model_context)

# trubrics model methods provide a range of helper funtions to apply the model to your dataset
trubrics_model.compute_performance_on_test_set()

## Init trubrics validator

In [ ]:
from trubrics.validators.base import Validator

In [ ]:
model_validator = Validator(trubrics_model=trubrics_model)

In [ ]:
model_validator.validate_performance_against_threshold(threshold=0.15)

In [ ]:
edge_case_data = X_test.iloc[0].to_dict()

In [ ]:
sec_validation = model_validator.validate_single_edge_case_in_range(
    edge_case_data=edge_case_data, lower_output=120, upper_output=140
)
print("Outcome: ", sec_validation.outcome)
print("Result: ", sec_validation.result)